In [ ]:
import numpy as np
import scipy.stats as sps
from sklearn.datasets import load_iris
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
import numpy as np
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.optim import optimize_acqf
from botorch.acquisition import UpperConfidenceBound
import numpy as np
from scipy.stats import norm
from botorch.utils import standardize


def objective(params):
    n = 10
    C, gamma = params
    X, y = load_iris(return_X_y=True)
    np.random.seed(0)
    indices = np.random.permutation(len(X))
    X_train, X_test, y_train, y_test = X[indices[:n]], X[indices[n:]], y[indices[:n]], y[indices[n:]]
    clf = SVC(C=C, gamma=gamma)
    clf.fit(X_train, y_train)
    return -clf.score(X_test, y_test)
    
# 假设 objective 是目标函数，acquisition 是获取函数，gp 是高斯过程模型
# bounds 是参数空间的边界，x_samples 和 y_samples 是已有的样本点和对应的函数值

# 设置贝叶斯优化循环的迭代次数
n_iter = 20

# 存储已有的样本点和对应的函数值
x_samples = [[0.01, 0.001]]  # 假设已经初始化
y_samples = [objective(x_samples[0])]  # 假设已经初始化

# 定义一个模拟的目标函数，这里我们用GP模型作为预测模型
kernel = DotProduct(sigma_0=0) + WhiteKernel(noise_level=1)
bounds=np.array([[1e-3, 1e-5],[1e+4, 5e-1]])
# C [1e-3, 1e3]
# gamma [1e-5, 1e-1]

# 贝叶斯优化循环
for i in range(n_iter):
    # 使用现有的样本点更新高斯过程模型
    x_samples = torch.as_tensor(x_samples, dtype=torch.float64)
    y_samples =  torch.as_tensor(y_samples, dtype=torch.float64)[:, None]

    x_samples = standardize(x_samples)
    y_samples = standardize(y_samples)

    gp = SingleTaskGP(x_samples,  y_samples)
    mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
    fit_gpytorch_mll(mll)
    
    UCB = UpperConfidenceBound(gp, beta=0.1)
    
    # acqf
    x_next, best_acq_value = optimize_acqf(
        UCB, bounds=torch.as_tensor(bounds, dtype=torch.float32), q=1, num_restarts=1, raw_samples=5,
    )
    
    # 打印当前迭代的下一个样本点和获取函数的值
    print(f"Iteration {i+1}: next sample is {x_next}, acquisition value is {best_acq_value}")
    
    # 在下一个样本点上评估目标函数，并将其添加到现有的样本点中
    y_next = objective(x_next.cpu().detach().numpy().tolist()[0])
    x_samples = np.vstack((x_samples, x_next))
    y_samples = np.append(y_samples, y_next)

import numpy as np

# 假设 x_samples 和 y_samples 是通过贝叶斯优化得到的样本点和对应的函数值

# 找到使目标函数值最小的索引
best_index = np.argmin(y_samples)

# 使用找到的索引获取最佳样本点
best_x = x_samples[best_index]

# 获取最佳样本点对应的函数值，即最佳准确率
best_y = y_samples[best_index]

# 打印最佳参数和最佳准确率
# 假设最佳参数是 C 和 gamma，且它们是样本点向量中的前两个元素
print(f"Best parameters: C={best_x[0]}, gamma={best_x[1]}")
print(f"Best accuracy: {best_y}")